In [1]:
import numpy as np
import itertools
np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
class Bar:
    def __init__(self, title, dia, length, number):
        self.title = title
        self.dia = dia
        self.length = length
        self.number = number

In [3]:
def get_coefficien_list(L, bar_length_list):
    bar_length_min = min(bar_length_list)
    max_iter = int(L/bar_length_min)+1
    co_efficient_list = np.arange(max_iter)

    product_arguments = [co_efficient_list]*len(bar_length_list)

    # print(product_arguments)
    return list(itertools.product(*product_arguments))

In [4]:
def get_coefficien_arr(L, bar_length_list):
    coefficien_arr = np.array(get_coefficien_list(L, bar_length_list))
    bar_length_arr = np.array(bar_length_list)

    sum_arr = np.sum(coefficien_arr*bar_length_arr, axis=1)
    # Remove all item which sum length = 0 or greater than L
    condition = (sum_arr == 0) | (sum_arr > L)
    condition = np.invert(condition)

    return coefficien_arr[condition]

In [5]:
def get_sorted_list(L,bar_length_list):
    coeff_arr = get_coefficien_arr(L,bar_length_list)
    westageArr =L-np.sum(coeff_arr * bar_length_list, axis=1)
    indexArr = np.argsort(westageArr)
    return coeff_arr[indexArr]

In [6]:
def get_bar_length_array(barList):
    arr =[]
    for x in barList:
        arr.append(x.length)
    return np.array(arr)

In [7]:
def get_bar_number_array(barList):
    arr =[]
    for x in barList:
        arr.append(x.number)
    return np.array(arr)

In [8]:
def get_results(L,bar_length_arr,bar_number_array):
    results = []
    sorted_arr = get_sorted_list(L,bar_length_arr)
    for comb in sorted_arr:
        if bar_number_array.sum()==0:
            break
        cond = bar_number_array!=0
        if comb.sum()==(comb*cond).sum():
            min_arr = bar_number_array/comb
            min_arr[min_arr==np.nan] = np.inf
#             print(min_arr)
            cut = int(np.nanmin(min_arr))
            
            if cut !=0:
                bar_number_array = bar_number_array - comb*cut
                westage = L-(bar_length_arr*comb).sum()
                result = {'cut':cut,'combination':comb,'westage_per_cut':westage,'total_westage':cut*westage,'L':L}
                results.append(result)
    
    return results

In [9]:
def get_distributed_list(L,bar_length_arr,bar_number_array):
    distribution_list = []
    results = get_results(L,bar_length_arr,bar_number_array)

    for x in results:
        if x['westage_per_cut']!=0 and x['westage_per_cut']>=bar_length_array.min():
            distribution_list.append(x)      
    return distribution_list;
    

In [10]:
def get_sub_cut(item,bar_length_array,bar_number_array):
#     print(item)
    l = item['westage_per_cut']
    #item['L'] = l
    total_cut = item['cut']
    bll = bar_length_array
    bna = bar_number_array
    
    results = get_results(l,bll,bna) # Find the Results of All Possible Cuts
    #Now Compare With Total Cuts
    fr = []
      
    for x in results:
        if x['cut']<=total_cut:
            total_cut= total_cut-x['cut']
            fr.append(x)
        else:
            x['cut']= total_cut
            x['total_westage'] = x['cut']*x['westage_per_cut']
            fr.append(x)
            total_cut= total_cut-x['cut']
            break 
    item['total_westage'] = item['westage_per_cut']*total_cut
    return fr

In [11]:
def get_process_list(L,bar_length_array,bar_number_array):
    results = get_results(L,bar_length_array,bar_number_array)
    distribution_list = get_distributed_list(L,bar_length_array,bar_number_array)
    if len(distribution_list)<=0:
        return results
#         print("distribution_list length 0") # here return Results Array
    else:
        return get_process_distribution_list(L,distribution_list,bar_length_array,bar_number_array)

In [12]:
def get_process_distribution_list(L,distribution_list,bar_length_array,bar_number_array):
    final_list = []
    for item in distribution_list:
        bar_number_array = bar_number_array-item['combination']*item['cut']
        item['total_westage'] =0
        item['length_arr'] = bar_length_array
        final_list.append(item)
        sub_cuts = get_sub_cut(item,bar_length_array,bar_number_array)
        for x in sub_cuts:
            final_list.append(x)
            bar_number_array = bar_number_array-x['combination']*x['cut']
    
    sorted_arr = get_sorted_list(L,bar_length_array)
    
    for x in sorted_arr:
        cond = bar_number_array ==0
        bal = x[cond]
        if bal.sum() == 0:
            if bar_number_array.sum()==0:
                break
            cond = bar_number_array!=0
            min_arr = bar_number_array/x
            cut = int(np.nanmin(min_arr))
            if cut !=0:
                bar_number_array = bar_number_array - x*cut
                westage = L-(bar_length_array*x).sum()
                result = {'cut':cut,'combination':x,'westage_per_cut':westage,'total_westage':cut*westage,'L':L}
                final_list.append(result)
#     print(bar_number_array)
    
    return final_list

In [13]:
def get_total_westage(results):
    total_westage = 0
    for x in results:
        total_westage= total_westage+x['total_westage']
    return total_westage

In [14]:
L = 12
# bar_length_list = [2.5, 3, 4]

# bar1 = Bar('A', 12, 1.5, 450)
# bar2 = Bar('B', 12, 2, 430)
# bar3 = Bar('C', 12, 4, 600)
# bar4 = Bar('D', 12, 5.5, 854)
# bar5 = Bar('E', 12, 7.5, 530)
# bar6 = Bar('F', 12, 8.5, 745)

bar1 = Bar('A', 12, 1.5, 1000)
bar2 = Bar('B', 12, 2.5, 2000)
bar3 = Bar('C', 12, 4, 328)
bar4 = Bar('D', 12, 6.5, 450)
bar5 = Bar('E', 12, 7.5, 530)
bar6 = Bar('F', 12, 8.5, 2000)

# bar1 = Bar('A', 12, 1.5, 300)
# bar2 = Bar('B', 12, 2, 700)
# bar3 = Bar('C', 12, 4, 600)
# bar4 = Bar('D', 12, 5.5, 210)
# bar5 = Bar('E', 12, 7.5, 530)
# bar6 = Bar('F', 12, 8.5, 555)

In [15]:
barList = [bar1,bar2,bar3,bar4,bar5,bar6]
bar_length_array = get_bar_length_array(barList)
bar_number_array = get_bar_number_array(barList)
final_list = get_process_list(L,bar_length_array,bar_number_array)
final_list

C:\Users\sohel\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
C:\Users\sohel\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


[{'cut': 903,
  'combination': array([0, 0, 0, 0, 0, 1]),
  'westage_per_cut': 3.5,
  'total_westage': 0.0,
  'L': 12,
  'length_arr': array([1.5, 2.5, 4. , 6.5, 7.5, 8.5])},
 {'cut': 500,
  'combination': array([2, 0, 0, 0, 0, 0]),
  'westage_per_cut': 0.5,
  'total_westage': 250.0,
  'L': 3.5},
 {'cut': 403,
  'combination': array([0, 1, 0, 0, 0, 0]),
  'westage_per_cut': 1.0,
  'total_westage': 403.0,
  'L': 3.5},
 {'cut': 530,
  'combination': array([0, 0, 0, 0, 1, 0]),
  'westage_per_cut': 4.5,
  'total_westage': 0.0,
  'L': 12,
  'length_arr': array([1.5, 2.5, 4. , 6.5, 7.5, 8.5])},
 {'cut': 328,
  'combination': array([0, 0, 1, 0, 0, 0]),
  'westage_per_cut': 0.5,
  'total_westage': 164.0,
  'L': 4.5},
 {'cut': 202,
  'combination': array([0, 1, 0, 0, 0, 0]),
  'westage_per_cut': 2.0,
  'total_westage': 404.0,
  'L': 4.5},
 {'cut': 450,
  'combination': array([0, 2, 0, 1, 0, 0]),
  'westage_per_cut': 0.5,
  'total_westage': 225.0,
  'L': 12},
 {'cut': 495,
  'combination': array

In [16]:
for x in final_list:
    bars = []
    
    for i,comb in enumerate(x['combination']):
        if comb !=0:
            bar = barList[i]
            b ={}
            b['title'] = bar.title
            b['dia'] = bar.dia
            b['cutting_bar_length'] = x['L']
            b['length'] = bar.length
            b['cut'] = x['cut']
            b['factor'] = comb
            b['number_of_bars']=b['cut']*b['factor']
            bars.append(b)
    x['bars'] = bars     
    
#     for bar in np.array(barList)[x['combination']!=0]:
#         b ={}
#         b['title'] = bar.title
#         b['dia'] = bar.dia
#         b['length'] = bar.length
#         bars.append(b)
#     x['cutting_bar_length'] = x['L']
#     x['bars'] = bars
    
final_list

[{'cut': 903,
  'combination': array([0, 0, 0, 0, 0, 1]),
  'westage_per_cut': 3.5,
  'total_westage': 0.0,
  'L': 12,
  'length_arr': array([1.5, 2.5, 4. , 6.5, 7.5, 8.5]),
  'bars': [{'title': 'F',
    'dia': 12,
    'cutting_bar_length': 12,
    'length': 8.5,
    'cut': 903,
    'factor': 1,
    'number_of_bars': 903}]},
 {'cut': 500,
  'combination': array([2, 0, 0, 0, 0, 0]),
  'westage_per_cut': 0.5,
  'total_westage': 250.0,
  'L': 3.5,
  'bars': [{'title': 'A',
    'dia': 12,
    'cutting_bar_length': 3.5,
    'length': 1.5,
    'cut': 500,
    'factor': 2,
    'number_of_bars': 1000}]},
 {'cut': 403,
  'combination': array([0, 1, 0, 0, 0, 0]),
  'westage_per_cut': 1.0,
  'total_westage': 403.0,
  'L': 3.5,
  'bars': [{'title': 'B',
    'dia': 12,
    'cutting_bar_length': 3.5,
    'length': 2.5,
    'cut': 403,
    'factor': 1,
    'number_of_bars': 403}]},
 {'cut': 530,
  'combination': array([0, 0, 0, 0, 1, 0]),
  'westage_per_cut': 4.5,
  'total_westage': 0.0,
  'L': 12,
 

In [17]:
get_total_westage(final_list)

4048.0

In [18]:
bar_length_array

array([1.5, 2.5, 4. , 6.5, 7.5, 8.5])

In [19]:
bar_number_array

array([1000, 2000,  328,  450,  530, 2000])

In [20]:
percenetage_westage = get_total_westage(final_list)*100/(bar_length_array*bar_number_array).sum()

In [21]:
percenetage_westage

12.764883955600403

In [22]:
len(final_list)

9

In [23]:
final_list[0]

{'cut': 903,
 'combination': array([0, 0, 0, 0, 0, 1]),
 'westage_per_cut': 3.5,
 'total_westage': 0.0,
 'L': 12,
 'length_arr': array([1.5, 2.5, 4. , 6.5, 7.5, 8.5]),
 'bars': [{'title': 'F',
   'dia': 12,
   'cutting_bar_length': 12,
   'length': 8.5,
   'cut': 903,
   'factor': 1,
   'number_of_bars': 903}]}

In [24]:
total =0
for x in final_list:
    if x['L']==12:
        total = total+x['cut']

In [25]:
total

2980